In [ ]:
import os
import cv2
from matplotlib import pyplot as plt


In [ ]:
os.chdir("c:\\Users\\smatysik\\projects\\nvimagecodec\\build\\example\\example_decode\\RelWithDebInfo")
#os.chdir("/mnt/c/Users/smatysik/projects/nvImageCodecsWSL2/build/example/example_decode")


# OpenCV

Load small image with OpenCV

In [ ]:
%%time 
cv_img_cat = cv2.imread("cat.j2k")


In [ ]:
cv_img_cat = cv2.cvtColor(cv_img_cat, cv2.COLOR_BGR2RGB)
plt.imshow(cv_img_cat)

Load 4k image with OpenCV

In [ ]:
%%time 
#%%timeit -r 1 -n 10
cv_img_4k = cv2.imread("4k_lossless.jp2")


In [ ]:
cv_img_4k = cv2.cvtColor(cv_img_4k, cv2.COLOR_BGR2RGB)
plt.imshow(cv_img_4k)


# nvImageCodecs

In [ ]:
import nvimgcodecs

Load small Jpeg2000 image with nvImageCodecs

In [ ]:
%%time
#%%timeit -r 5 -n 10
nv_img_cat = nvimgcodecs.imread("cat.j2k")


Load 4k Jpeg2000 (in jp2 container) image with nvImageCodecs

In [ ]:
%%time
#%%timeit -r 5 -n 10
nv_img4k = nvimgcodecs.imread("4k_lossless.jp2")

Save 4k image to bmp file with nvImageCodecs

In [10]:
%%time
#%%timeit -r 1 -n 10
nvimgcodecs.imwrite("4k_lossless-jp2-o.bmp", nv_img4k)
nvimgcodecs.imwrite("4k_lossless-jp2-o.jpg", nv_img4k)


Read back with OpenCV just save (with nvImageCodecs) bmp image 

In [ ]:
cv_img_4k_bmp = cv2.imread("4k_lossless.bmp")


In [ ]:
cv_img_4k_bmp = cv2.cvtColor(cv_img_4k_bmp, cv2.COLOR_BGR2RGB)
plt.imshow(cv_img_4k_bmp)


Save as Jpeg 2000

In [ ]:
%%time
# %%timeit -r 1 -n 10
nvimgcodecs.imwrite("new_4k_lossless.j2k", nv_img4k)


## nvImageCodes Image  __cuda_array_interface__ support

In [ ]:
print(nv_img4k.__cuda_array_interface__)
print(nv_img4k.shape)


Lets test it with cupy

In [ ]:

import cupy as cp

Convert to Numpy (CPU) array

In [ ]:

np_img4k = cp.asnumpy(nv_img4k)



In [ ]:
plt.imshow(np_img4k)

In [ ]:
%%time
nv_img_jpg = nvimgcodecs.imread("cat_q95_444.jpg")
print(nv_img_jpg.__cuda_array_interface__)
np_img4k = cp.asnumpy(nv_img_jpg)
plt.imshow(np_img4k)


Convert tp cupy ndarray

In [ ]:
%%time
cp_img4k = cp.asarray(nv_img4k)


Lets do some opration on image in GPU using cupyx.scipy

In [ ]:
import cupyx.scipy.ndimage


In [ ]:

cp_img_4k_rotated = cupyx.scipy.ndimage.rotate(cp_img4k, 90)
cp_img_4k_gaussian = cupyx.scipy.ndimage.gaussian_filter(cp_img4k, sigma = 15)
cp_img_4k_laplace = cupyx.scipy.ndimage.laplace(cp_img4k)
cp_img_4k_single_channel = cp_img4k[:, :, 1]

print(cp_img_4k_rotated.__cuda_array_interface__)


In [ ]:
np_img_4k = cp.asnumpy(cp_img_4k_rotated)
plt.imshow(np_img_4k)


In [ ]:
np_img_4k = cp.asnumpy(cp_img_4k_gaussian)
plt.imshow(np_img_4k)


Save rotated version which is in cupy ndarray format (GPU) to file using nvImageCodecs 

In [ ]:
nv_rotated_img = nvimgcodecs.asimage(cp_img_4k_rotated)


In [ ]:
%%time
nvimgcodecs.imwrite("rotated.bmp", nv_rotated_img)


Read and show saved rotaed image to verify

In [ ]:
cv_img_4k_bmp = cv2.imread("rotated.bmp")
cv_img_4k_bmp = cv2.cvtColor(cv_img_4k_bmp, cv2.COLOR_BGR2RGB)
plt.imshow(cv_img_4k_bmp)


Save as JPEG2000

nvJpeg2000 now supports only Planar RGB and cupy ndarray is in interleaved RGB order so we need to convert it before providing buffer to nvJpeg2000. In future nvJpeg2000 should probably support I_RGB.

In [ ]:
%%time
cp_img_4k_rotated_planar = cp.moveaxis(cp_img_4k_rotated, (0, 1), (1, 0))
cp_img_4k_rotated_planar = cp.asfortranarray(cp_img_4k_rotated_planar)
cp_img_4k_rotated_planar = cp.moveaxis(cp_img_4k_rotated_planar, (0,1), (1,0))



In [ ]:
%%time
nv_rotated_img = nvimgcodecs.asimage(cp_img_4k_rotated_planar)


In [ ]:
%%time
nvimgcodecs.imwrite("rotated.j2k", nv_rotated_img)


Interoperability with cuCIM (only Linux)

In [ ]:
from cucim.skimage.filters import gaussian


In [ ]:
cp_img4k_blurred = gaussian(cp_img_4k_single_channel, sigma=5)
cp_img4k_blurred_color = gaussian(nv_img4k, sigma=25)
image = cp.asnumpy(cp_img4k_blurred)
plt.imshow(image, cmap='gray')


In [ ]:
from cucim.skimage.filters import threshold_otsu
# determine threshold
threshold = threshold_otsu(cp_img4k_blurred)

# binarize image by apply the threshold
cp_binary_gpu = cp_img4k_blurred > threshold

image = cp.asnumpy(cp_binary_gpu)
plt.imshow(image, cmap='gray')


In [ ]:
from cucim.skimage.morphology import binary_erosion, disk

eroded_gpu = binary_erosion(cp_binary_gpu, selem=disk(2))
image = cp.asnumpy(eroded_gpu)
plt.imshow(image, cmap='gray')


In [ ]:
print(cp_img4k_blurred_color.__cuda_array_interface__)

In [ ]:
%%time
cp_img_planar = cp.moveaxis(cp_img4k_blurred_color, (0, 1), (1, 0))
cp_img_planar = cp.asfortranarray(cp_img_planar)
cp_img_planar = cp.moveaxis(cp_img_planar, (0, 1), (1, 0))
nv_img = nvimgcodecs.asimage(cp_img_planar)


In [ ]:
%%time
nvimgcodecs.imwrite("blurred.j2k", nv_img)
